In [8]:
# imports

import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
# constants

path = "./dataset.csv"

In [10]:
# read data

data = pd.read_csv(path, header = None, names = ['aoa', 'Re', 'mc', 'mcp', 'th', 'up', 'lp'])

In [11]:
y = data[['up', 'lp']]
y_u = data['up']
y_u_sep = (data['up']<1).astype(int)
#y_l = data['lp']
X = data.drop(['up', 'lp'], axis = 1)


In [12]:
X_u_train, X_u_test, y_u_train, y_u_test = train_test_split(X, y_u, test_size=0.33, random_state=42)
#X_l_train, X_l_test, y_l_train, y_l_test = train_test_split(X, y_l, test_size=0.33, random_state=42)
_, _, y_u_sep_train, y_u_sep_test = train_test_split(X, y_u_sep, test_size=0.33, random_state=42)

In [13]:
X_u_train.values[0]

array([6.78557714e+00, 4.45195267e+07, 4.35486475e-01, 1.03688921e+01,
       3.23562531e+01])

In [14]:
#X_l_train.values[0]

In [15]:
y_u_sep_train[4]

1

In [16]:
# Quick baseline

model_u = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=1, colsample_bytree=1)
model_l = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=1, colsample_bytree=1)

classify_model = xgb.XGBClassifier()


In [18]:
model_u.fit(X_u_train, y_u_train)

#model_l.fit(X_l_train, y_l_train)

classify_model.fit(X_u_train, y_u_sep_train)

[09:45:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBClassifier()

In [19]:
def loss(y, yhat):
    return np.mean(np.sqrt((y-yhat)**2))

In [20]:
y_u_sep_hat = classify_model.predict(X_u_test)

In [21]:
accuracy_score(y_u_sep_hat, y_u_sep_test)

0.9703087885985748

In [22]:
y_u_hat = model_u.predict(X_u_test)
#y_l_hat = model_l.predict(X_l_test)

In [23]:
loss(y_u_hat, y_u_test)

0.045001430228494274

In [29]:
# Let's give it an extra boost

y_u_test

56      0.001087
194     1.000000
2226    0.930579
233     1.000000
1902    1.000000
          ...   
672     1.000000
2355    1.000000
1054    1.000000
1711    1.000000
309     0.002827
Name: up, Length: 842, dtype: float64

In [30]:
y_u_sep_test

56      1
194     0
2226    1
233     0
1902    0
       ..
672     0
2355    0
1054    0
1711    0
309     1
Name: up, Length: 842, dtype: int64

In [34]:
y_u_sep_hat

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [42]:
# Now we combine the greater certainty of the classifier to fix regression 1's values

def boost_predictions(regression_preds, classification_preds):
    return ((regression_preds-1)*classification_preds)+1

In [43]:
y_u_hat_boosted = boost_predictions(y_u_hat, y_u_sep_hat)

In [44]:
loss(y_u_hat_boosted, y_u_test)

0.0380903556144919